<a href="https://colab.research.google.com/github/18708064/big-data/blob/master/postblock2_q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Question 3.1

In [ ]:
# Map function
def map(filename, content):
    # For each character in the content of the file
    for char in content:
        # Emit the character as the key and the value as 1
        EmitIntermediate(char, 1)

# Reduce function
def reduce(char, values):
    # Sum the counts for each character key
    result = sum(values)
    Emit(char, result)


Question 3.5

In [ ]:
# Combiner function
def combiner(char, values):
    # Partially sum values before sending to reducer
    partial_sum = sum(values)
    Emit(char, partial_sum)


Question 4

In [ ]:
def mapper(hostname, document):
    terms = document.split()
    term_counts = Counter(terms)
    for term, count in term_counts.items():
        if count > 1:
            yield (hostname, (term, count))


def reducer(hostname, term_vectors):
    term_frequency = Counter()

    # Sum up term frequencies
    for term, count in term_vectors:
        term_frequency[term] += count

    # Emit terms that occur at least twice
    filtered_term_vector = {term: count for term, count in term_frequency.items() if count >= 2}

    yield (hostname, filtered_term_vector)


Question 4.2


In [3]:
def mapper(hostname, document):
    terms = document.split()  # Split the document into words (tokens)
    term_counts = Counter(terms)  # Count the occurrences of each term
    for term, count in term_counts.items():
        if count > 1:  # Only emit terms that appear more than once
            yield (hostname, (term, count))



In [4]:
def reducer(hostname, term_vectors):
    term_frequency = Counter()  # Initialize counter to store term frequencies

    # Aggregate the term frequencies
    for term, count in term_vectors:
        term_frequency[term] += count

    # Filter and emit terms that occur at least twice
    filtered_term_vector = {term: count for term, count in term_frequency.items() if count >= 2}

    yield (hostname, filtered_term_vector)

